In [1]:
%matplotlib inline


# Numerical libraries
import numpy as np   

# Import Linear Regression machine learning library
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score

# to handle data in form of rows and columns 
import pandas as pd    

# importing ploting libraries
import matplotlib.pyplot as plt   

#importing seaborn for statistical plots
import seaborn as sns

In [4]:
mpg_df = pd.read_csv("d:\\ML_Data\\car-mpg.csv")  
mpg_df = mpg_df.drop('car_name', axis=1)
mpg_df['origin'] = mpg_df['origin'].replace({1: 'america', 2: 'europe', 3: 'asia'})
mpg_df = pd.get_dummies(mpg_df, columns=['origin'])
mpg_df = mpg_df.replace('?', np.nan)
mpg_df = mpg_df.apply(lambda x: x.fillna(x.median()),axis=0)

# separate independent and dependent variables

In [5]:
# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = mpg_df.drop('mpg', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = mpg_df[['mpg']]


In [6]:
from sklearn import preprocessing

# scale all the columns of the mpg_df. This will produce a numpy array
X_scaled = preprocessing.scale(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)  # ideally the training and test should be 

In [7]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.30, random_state=1)

# fit a simple linear model

In [8]:
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

The coefficient for cyl is 2.5059518049385052
The coefficient for disp is 2.5357082860560483
The coefficient for hp is -1.7889335736325294
The coefficient for wt is -5.551819873098725
The coefficient for acc is 0.11485734803440854
The coefficient for yr is 2.931846548211609
The coefficient for car_type is 2.977869737601944
The coefficient for origin_america is -0.5832955290166003
The coefficient for origin_asia is 0.3474931380432235
The coefficient for origin_europe is 0.3774164680868855


# Create a regularized RIDGE model and note the coefficients

In [9]:
ridge = Ridge(alpha=.3)
ridge.fit(X_train,y_train)
print ("Ridge model:", (ridge.coef_))
    

Ridge model: [[ 2.47057467  2.44494419 -1.78573889 -5.47285499  0.10115618  2.92319984
   2.94492098 -0.57949986  0.34667456  0.37344909]]


# Create a regularized LASSO model and note the coefficients

In [10]:
lasso = Lasso(alpha=0.2)
lasso.fit(X_train,y_train)
print ("Lasso model:", (lasso.coef_))

# Observe, many of the coefficients have become 0 indicating drop of those dimensions from the model

Lasso model: [ 0.          0.         -0.3475456  -4.01181473  0.          2.64248634
  1.07111166 -0.54724128  0.          0.        ]


## Let us compare their scores

In [11]:
print(regression_model.score(X_train, y_train))
print(regression_model.score(X_test, y_test))


0.8513421387780066
0.8343770256960538


In [12]:
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))

0.8518882171608504
0.8343617931312615


In [13]:
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))

0.8547810865027448
0.8114389394513553


In [14]:
# More or less similar results but with less complex models.  Complexity is a function of variables and coefficients
## Note - with Lasso, we get equally good result in test though not so in training.  Further, the number of dimensions is much less
# in LASSO model than ridge or un-regularized model

# Let us generate polynomial models reflecting the non-linear interaction between some dimensions

In [15]:
from sklearn.preprocessing import PolynomialFeatures

In [16]:
poly = PolynomialFeatures(degree = 2, interaction_only=True)

#poly = PolynomialFeatures(2)

In [17]:
X_poly = poly.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.30, random_state=1)
X_train.shape

(278, 56)

# Fit a simple non regularized linear model on poly features-

In [18]:
regression_model.fit(X_train, y_train)
print(regression_model.coef_[0])


[-9.67853872e-13 -1.06672046e+12 -4.45865268e+00 -2.24519565e+00
 -2.96922206e+00 -1.56882955e+00  3.00019063e+00 -1.42031640e+12
 -5.46189566e+11  3.62350196e+12 -2.88818173e+12 -1.16772461e+00
 -1.43814087e+00 -7.49492645e-03  2.59439087e+00 -1.92409515e+00
 -3.41759793e+12 -6.27534905e+12 -2.44065576e+12 -2.32961194e+12
  3.97766113e-01  1.94046021e-01 -4.26086426e-01  3.58203125e+00
 -2.05296326e+00 -7.51019934e+11 -6.18967069e+11 -5.90805593e+11
  2.47863770e-01 -6.68518066e-01 -1.92150879e+00 -7.37030029e-01
 -1.01183732e+11 -8.33924574e+10 -7.95983063e+10 -1.70394897e-01
  5.25512695e-01 -3.33097839e+00  1.56301740e+12  1.28818991e+12
  1.22958044e+12  5.80200195e-01  1.55352783e+00  3.64527008e+11
  3.00431724e+11  2.86762821e+11  3.97644043e-01  8.58604718e+10
  7.07635073e+10  6.75439422e+10 -7.25449332e+11  1.00689540e+12
  9.61084146e+11  2.18532428e+11 -4.81675252e+12  2.63818648e+12]


In [19]:
ridge = Ridge(alpha=.3)
ridge.fit(X_train,y_train)
print ("Ridge model:", (ridge.coef_))

Ridge model: [[ 0.          3.73512981 -2.93500874 -2.13974194 -3.56547812 -1.28898893
   3.01290805  2.04739082  0.0786974   0.21972225 -0.3302341  -1.46231096
  -1.17221896  0.00856067  2.48054694 -1.67596093  0.99537516 -2.29024279
   4.7699338  -2.08598898  0.34009408  0.35024058 -0.41761834  3.06970569
  -2.21649433  1.86339518 -2.62934278  0.38596397  0.12088534 -0.53440382
  -1.88265835 -0.7675926  -0.90146842  0.52416091  0.59678246 -0.26349448
   0.5827378  -3.02842915 -0.36548074  0.5956112  -0.15941014  0.49168856
   1.45652375 -0.43819158 -0.20964198  0.77665496  0.36489921 -0.4750838
   0.3551047   0.23188557 -1.42941282  2.06831543 -0.34986402 -0.32320394
   0.39054656  0.06283411]]


In [20]:
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))


0.8613398053698552
0.9143225702003366


In [21]:
lasso = Lasso(alpha=0.1)
lasso.fit(X_train,y_train)
print ("Lasso model:", (lasso.coef_))


Lasso model: [ 0.         -0.         -0.         -1.59613165 -5.22452383 -0.
  2.86907439  0.03030592 -0.10514919  0.          0.         -0.
 -0.          0.          0.28971732 -0.          0.         -0.
  0.11457443 -0.          0.          1.15720495  0.          0.
 -0.          0.          0.         -0.          0.04724906  0.
 -0.6925298  -0.          0.          0.         -0.         -0.
 -0.         -0.67082659  0.         -0.         -0.          0.16918498
 -0.         -0.61771612  0.          0.36046427  0.         -0.37086554
  0.          0.         -0.         -0.          0.18165859 -0.
 -0.         -0.        ]


In [22]:
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))


0.8802228448476971
0.8900519684208552
